## Initiez-vous à la statistique inférentielle
   
### Taux de guérison suite à un nouveau traitement.

Considérons ce premier cas, qu’on peut qualifier de discret :
Un laboratoire cherche à savoir si la nouvelle composition du médicament (contre une maladie bénigne) qu’il compte 
commercialiser améliore le taux de guérison par rapport à un médicament déjà sur le marché. Des tests cliniques ont été 
effectués sur n=216 (n = l'echantillon)
individus sur lesquels on a observé la guérison (notée 1 ) ou la non-guérison (notée 0) : {x1,…,x216}={1,0,1,1,…,1,0}
xi désigne l’observation de la guérison ou non pour l’individu i.
Le laboratoire observe au total 167 guérisons, soit environ 77,3% de guérisons.
Le laboratoire s’adresse à un data analyst pour répondre à plusieurs de ses interrogations :

-  Il aimerait connaître le taux de guérison (théorique)p suite à la prise de son médicament.
-  Le laboratoire étant conscient que le taux de guérison théorique sera délicat à appréhender parfaitement, 
il souhaiterait disposer d’une fourchette de ce taux de guérison.
-  Enfin, il aimerait vérifier que son nouveau médicament est (significativement) meilleur que celui actuellement sur la 
marché dont le taux de guérison avéré est p0=0.75 (75%). 

## Statistique inférentielle

### IC sur Guerison 

On importe le fichier contenant les guérisons ou non-guérisons :

In [1]:
import pandas as pd
import numpy as np, math
guerison =pd.read_csv("guerison.txt")

In [2]:
guerison

,guerison
0,1
1,0
2,1
3,1
4,1
...,...
211,1
212,1
213,0
214,1


On peut estimer le taux de guerison théorique  p :

In [3]:
n_guerison=len(guerison)
n_guerison_gueris= sum(guerison[guerison["guerison"]==1]["guerison"])
p_estim= n_guerison_gueris/float(n_guerison)

Si on souhaite encadrer le taux de guérison  p
  avec une probabilité de  1−α=95%
 , on obtient alors comme intervalle de confiance ( Φ1−α2=Φ0.975≃1.96
 ) :

[167/216−1.96167216(1−167/216)216‾‾‾‾‾‾‾‾‾√;167/216+1.96167216(1−167216)216‾‾‾‾‾‾‾‾‾√]
 .

Si on lance “manuellement” les calculs au niveau de test 5% :

In [4]:
alpha = 0.05
from scipy.stats import norm
icinf =p_estim-norm.ppf(1-alpha/2)*math.sqrt(p_estim*(1-p_estim)/n_guerison)
round(icinf,2)

0.72

In [5]:
icsup = p_estim+norm.ppf(1-alpha/2)*math.sqrt(p_estim*(1-p_estim)/n_guerison)
round(icsup,2)

0.83

On obtient alors : [0.72 ; 0.82]=[72% ; 83%] .

On constate que la largeur de l’intervalle n’est pas négligeable, mais n’oublions pas qu’il n’y a que 216 individus dans l’échantillon.

En pratique, le data analyst peut obtenir simplement cet intervalle à l’aide de la commande proportion_confint :

In [6]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(n_guerison_gueris,n_guerison, alpha=alpha, method='normal')

(0.7172980758199328, 0.8289982204763635)

On aurait également pu obtenir un intervalle de confiance “exact”, basé sur la loi binomiale :

In [7]:
'''
from statsmodels.stats.proportion import proportion_confint
proportion_confint(n_guerison_gueris,n_guerison, alpha=alpha, method='binom_test')
'''
# on observe une erreur lorsqu'on lance cette ligne de code sur l'argument 'binom_test'

"\nfrom statsmodels.stats.proportion import proportion_confint\nproportion_confint(n_guerison_gueris,n_guerison, alpha=alpha, method='binom_test')\n"

On constate que les résultats diffèrent très peu de l’intervalle de confiance asymptotique.

Enfin, si on avait choisi un niveau de confiance plus faible,  1−α=90%
  par exemple, on aurait obtenu un intervalle de confiance plus étroit :

In [8]:
from statsmodels.stats.proportion import proportion_confint
proportion_confint(n_guerison_gueris,n_guerison, alpha=0.1, method='normal')

(0.7262772899287175, 0.8200190063675787)

On obtient ici  [0.726;0.820]=[72.6%;82.0%]
 , la largeur de l’intervalle a bel et bien diminué !

### test sur Guerison

On teste :  {H0:p=p0H1:p>p0
 
avec  p0
 =0.75.

On considère que l’hypothèse gaussienne est acceptable ici, en effet :  np0(1−p0)=216(0.75)(1−0.75)=40.5>5
  .

Pour  α=5%
 , on a :

Φ1−α≃1.64
   p0+Φ1−αp0(1−p0)n‾‾‾‾‾‾‾√≃0.798.
 
On ne rejette pas H0 au niveau de test 5% car :  x¯=0.773≤0.798≃p0+Φ1−αp0(1−p0)n‾‾‾‾‾‾‾√.
 
La p-valeur vaut :  p−valeur=1−Φ(n‾√x¯−p0p0(1−p0)√)≃1−Φ(0.786)≃0.216.
 
On constate que le niveau de test retenu devrait au moins être égal à 21,6% pour rejeter H0 ! Accepter de se tromper dans plus de 20% des cas lorsque rejette l’hypothèse nulle n’est pas rencontré en pratique.

Au final, le laboratoire ne peut pas conclure avec un niveau de test raisonnable que le nouveau médicament est meilleur que celui déjà sur le marché.

In [9]:
import statsmodels
statsmodels.stats.proportion.binom_test(n_guerison_gueris,n_guerison, prop=0.75, alternative='larger')

0.24183479554060167